In [1]:
!pip3 install torch torchvision

    100% |████████████████████████████████| 591.8MB 25kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x626ae000 @  0x7ff6f1a062a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 23.7MB/s 
    100% |████████████████████████████████| 2.0MB 5.8MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
#파이토치
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

#사이킷런
from sklearn.datasets import load_digits
from sklearn import datasets, model_selection

#판다스
import pandas as pd

#맷플랏
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline

import numpy as np

## 데이터로드
mnist 데이터를 읽어 들인 다음, 화면에 출력

In [0]:
train_df = pd.read_csv('sample_data/mnist_train_small.csv')
test_df = pd.read_csv('sample_data/mnist_test.csv')

In [4]:
train_df.tail()

,6,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.581,0.582,0.583,0.584,0.585,0.586,0.587,0.588,0.589,0.590
19994,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19995,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19996,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19997,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19998,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
train_df.shape

(19999, 785)

In [0]:
train_Y = train_df["6"]
train_X = train_df.drop("6", axis=1)

In [7]:
train_X.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.581,0.582,0.583,0.584,0.585,0.586,0.587,0.588,0.589,0.590
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
test_Y = test_df["7"]
test_X = test_df.drop("7", axis=1)

## 텐서 생성

In [0]:
train_X = torch.from_numpy(train_X.as_matrix()).float()
train_Y = torch.from_numpy(train_Y.as_matrix()).long()

test_X = torch.from_numpy(test_X.as_matrix()).float()
test_Y = torch.from_numpy(test_Y.as_matrix()).long()

In [32]:
print(train_X.shape)
print(train_Y.shape)

torch.Size([19999, 784])
torch.Size([19999])


In [33]:
train = TensorDataset( train_X, train_Y)
print(train[0])

(tensor([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,  

In [0]:
#미니 배치로 분활
train_loader = DataLoader(train, batch_size=100, shuffle=True)

## 모델

In [0]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 256)
    self.fc3 = nn.Linear(256, 256)
    self.fc4 = nn.Linear(256, 128)
    self.fc5 = nn.Linear(128, 128)
    self.fc6 = nn.Linear(128, 10)
    
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = F.relu(self.fc5(x))
    x = F.dropout(x, training=self.training)
    x = self.fc6(x)
    return F.log_softmax(x)  

In [0]:
model = Net()

## 학습

In [0]:
#오차함수
loss_fuction = nn.CrossEntropyLoss()

In [0]:
#최적화
optimizer = optim.SGD(model.parameters(),lr=0.01)

In [0]:
# 학습 시작
for epoch in range(1000):
  total_loss = 0
  
  #분할해둔 데이터를 꺼내옴
  for train_x, train_y in train_loader:
    #계산 그래프 구성
    train_x, train_y = Variable(train_x), Variable(train_y)
    #경사 초기화
    optimizer.zero_grad()
    #순전파 계산
    output = model(train_x)
    #오차 계산 
    loss = loss_fuction(output, train_y)
    #역전파 계산
    loss.backward()
    #가중치 업데이트
    optimizer.step()
    #누적 오차 계산
    total_loss += loss.item()
  
  #100회 반복마다 누적 오차 출력
  if (epoch + 1) % 100 == 0:
    print(epoch+1, total_loss)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


100 0.11140666848041292
200 0.035533843869529846
300 0.018661608810816688
400 0.018825674997430042
500 0.03705702139177447
